# Regression Project: Predicting the Sale Prices of Homes in Seattle, Washington

Dataset is downloaded from Kaggle: https://www.kaggle.com/datasets/samuelcortinhas/house-price-prediction-seattle/code

In [2]:
# Importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20, 10)


In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [4]:
df_train.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price
0,3,2.5,2590.0,sqft,6000.00,sqft,98144,795000.0
1,4,2.0,2240.0,sqft,0.31,acre,98106,915000.0
2,4,3.0,2040.0,sqft,3783.00,sqft,98107,950000.0
3,4,3.0,3800.0,sqft,5175.00,sqft,98199,1950000.0
4,2,2.0,1042.0,sqft,NaN,NaN,98102,950000.0


In [12]:
df_train.shape, df_test.shape

((2016, 8), (505, 8))

In [13]:
df_train.info(), df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   beds            2016 non-null   int64  
 1   baths           2016 non-null   float64
 2   size            2016 non-null   float64
 3   size_units      2016 non-null   object 
 4   lot_size        1669 non-null   float64
 5   lot_size_units  1669 non-null   object 
 6   zip_code        2016 non-null   int64  
 7   price           2016 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 126.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   beds            505 non-null    int64  
 1   baths           505 non-null    float64
 2   size            505 non-null    float64
 3   size_units      505 non-null    object 
 4   l

(None, None)

## Exploring the dataset

Note that:
- beds = number of bedrooms in the house
- baths = number of bathrooms in the house and 0.5 means a half-bathroom which has a sink and a toilet but no shower or bathtub
- size = the size of the house in square feet
- size_units = the unit of the size of the house
- lot_size = the total area of the land where the property is located on. The lot belongs to the house owner.
- lot_size_units = the unit of the lot size
- zip_code = the zip code of the location of the house
- price = the price of the house in dollars (USD)
- 1 acre = 43560 square feet


In [15]:
# Checking for missing values
df_train.isnull().sum(), df_test.isnull().sum()

(beds                0
 baths               0
 size                0
 size_units          0
 lot_size          347
 lot_size_units    347
 zip_code            0
 price               0
 dtype: int64,
 beds               0
 baths              0
 size               0
 size_units         0
 lot_size          77
 lot_size_units    77
 zip_code           0
 price              0
 dtype: int64)

In [16]:
# Checking for the proportion of missing values in the dataset
df_train.isnull().sum()/df_train.shape[0]*100, df_test.isnull().sum()/df_test.shape[0]*100

(beds               0.000000
 baths              0.000000
 size               0.000000
 size_units         0.000000
 lot_size          17.212302
 lot_size_units    17.212302
 zip_code           0.000000
 price              0.000000
 dtype: float64,
 beds               0.000000
 baths              0.000000
 size               0.000000
 size_units         0.000000
 lot_size          15.247525
 lot_size_units    15.247525
 zip_code           0.000000
 price              0.000000
 dtype: float64)

Huge percentage of missing values in the lot_size column as well as the corresponding lot_size_units column in both datasets. We will assume that the minimum lot size  will be the size_unit, which is what we will use to fill it. This helps to logically ensure that the lot size is not less than the size of the house and deviation from the truth is minimal.

In [17]:
df_train[['size', 'size_units', 'lot_size', 'lot_size_units']].head()

,size,size_units,lot_size,lot_size_units
0,2590.0,sqft,6000.00,sqft
1,2240.0,sqft,0.31,acre
2,2040.0,sqft,3783.00,sqft
3,3800.0,sqft,5175.00,sqft
4,1042.0,sqft,NaN,NaN


In [20]:
df_train.groupby('lot_size_units')['lot_size_units'].count()

lot_size_units
acre     220
sqft    1449
Name: lot_size_units, dtype: int64

In [21]:
df_test.groupby('lot_size_units')['lot_size_units'].count()

lot_size_units
acre     59
sqft    369
Name: lot_size_units, dtype: int64

## Data Cleaning and Preprocessing

In [22]:
# Some values in the lot_size_units column are in acres and some are in sqft. We will convert all the values to sqft

# We will define a function to convert the values in the lot_size column to sqft
def convert_to_sqft(row):
    if row['lot_size_units'] == 'sqft':
        return row['lot_size']
    else:
        return row['lot_size']*43560

In [23]:
# Applying the function to the dataset
df_train['lot_size'] = df_train.apply(lambda row: convert_to_sqft(row), axis=1)

In [25]:
df_train.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price
0,3,2.5,2590.0,sqft,6000.0,sqft,98144,795000.0
1,4,2.0,2240.0,sqft,13503.6,acre,98106,915000.0
2,4,3.0,2040.0,sqft,3783.0,sqft,98107,950000.0
3,4,3.0,3800.0,sqft,5175.0,sqft,98199,1950000.0
4,2,2.0,1042.0,sqft,NaN,NaN,98102,950000.0


In [26]:
df_test['lot_size'] = df_test.apply(lambda row: convert_to_sqft(row), axis=1)

In [27]:
df_test.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price
0,3,3.0,2850.0,sqft,4200.0,sqft,98119,1175000.0
1,4,5.0,3040.0,sqft,5002.0,sqft,98106,1057500.0
2,3,1.0,1290.0,sqft,6048.0,sqft,98125,799000.0
3,3,2.0,2360.0,sqft,12196.8,acre,98188,565000.0
4,3,3.5,1942.0,sqft,1603.0,sqft,98107,1187000.0


In [28]:
# Dealing with missing values

#1. Finding the average lot_size:size and applying it to the missing values
df_train['avg_lot_ratio'] = df_train['lot_size']/df_train['size']
df_train.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price,avg_lot_ratio
0,3,2.5,2590.0,sqft,6000.0,sqft,98144,795000.0,2.316602
1,4,2.0,2240.0,sqft,13503.6,acre,98106,915000.0,6.028393
2,4,3.0,2040.0,sqft,3783.0,sqft,98107,950000.0,1.854412
3,4,3.0,3800.0,sqft,5175.0,sqft,98199,1950000.0,1.361842
4,2,2.0,1042.0,sqft,NaN,NaN,98102,950000.0,NaN


In [29]:
df_train['avg_lot_ratio'].mean()

38.00439044304304

In [30]:
df_train['avg_lot_ratio'].median()

2.334375

In [31]:
df_train['avg_lot_ratio'].max()

43560.0